https://colab.research.google.com/github/csaybar/EEwPython/blob/master/9_SpecializedAlgorithms.ipynb#scrollTo=tnChq4QoIPXE

In [ ]:
import ee
import geemap
import folium

In [ ]:
# EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
#     :param center: Center of the map (Latitude and Longitude).
#     :param dicc: Earth Engine Geometries or Tiles dictionary
#     :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
#     :zoom_start: Initial zoom level for the map.
#     :return: A folium.Map object.
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
        if ee.image.Image in [type(x) for x in v.values()]:
                folium.TileLayer(
                    tiles = v["tile_fetcher"].url_format,
                    attr  = 'Google Earth Engine',overlay =True,
                    name  = k
                ).add_to(mapViz)
        else:
            folium.GeoJson(
               data = v,
               name = k
               ).add_to(mapViz)
            mapViz.add_child(folium.LayerControl())
    return mapViz

In [ ]:
# Define a region of interest as a point.  Change the coordinates
# to get a classification of any place where there is imagery.
roi = ee.Geometry.Point(-122.3942, 37.7295)

In [ ]:
# Load Landsat 5 input imagery.
# 1. Filter to get only one year of images.
# 2. Filter to get only images under the region of interest.
# 3. Sort by scene cloudiness, ascending.
# 4. Get the first (least cloudy) scene.
landsat = ee.Image(ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA')\
                     .filterDate('2011-01-01', '2011-12-31')\
                     .filterBounds(roi)\
                     .sort('CLOUD_COVER')\
                     .first()
                  )

In [ ]:
# Compute cloud score.
cloudScore = ee.Algorithms.Landsat.simpleCloudScore(landsat).select('cloud')

In [ ]:
# Mask the input for clouds.  Compute the min of the input mask to mask
# pixels where any band is masked.  Combine that with the cloud mask.
Input = landsat.updateMask(landsat.mask().reduce('min').And(cloudScore.lte(50)))

In [ ]:
# Use MODIS land cover, IGBP classification, for training.
modis = ee.Image('MODIS/051/MCD12Q1/2011_01_01')\
          .select('Land_Cover_Type_1')

In [ ]:
# Sample the input imagery to get a FeatureCollection of training data.
training = Input.addBands(modis).sample(numPixels=5000, seed=0)

In [ ]:
# Make a Random Forest classifier and train it.
classifier = ee.Classifier.randomForest(10)\
               .train(training, 'Land_Cover_Type_1')

In [ ]:
# Classify the input imagery.
classified = Input.classify(classifier)

In [ ]:
# Get a confusion matrix representing resubstitution accuracy.
trainAccuracy = classifier.confusionMatrix()
print('Resubstitution error matrix: ', trainAccuracy.getInfo())
print('Training overall accuracy: ', trainAccuracy.accuracy().getInfo())

In [ ]:
# Sample the input with a different random seed to get validation data.
validation = Input.addBands(modis)\
                  .sample(numPixels=5000,seed=1)\
                  .filter(ee.Filter.neq('B1', None)) # Filter the result to get rid of any null pixels.

In [ ]:
# Classify the validation data.
validated = validation.classify(classifier)

In [ ]:
# Get a confusion matrix representing expected accuracy.
testAccuracy = validated.errorMatrix('Land_Cover_Type_1', 'classification')
print('Validation error matrix: ', testAccuracy.getInfo())
print('Validation overall accuracy: ', testAccuracy.accuracy().getInfo())

In [ ]:
# Define a palette for the IGBP classification.
igbpPalette = ['aec3d4', # water
               '152106', '225129', '369b47', '30eb5b', '387242', # forest
               '6a2325', 'c3aa69', 'b76031', 'd9903d', '91af40',  # shrub, grass
               '111149', # wetlands
               'cdb33b', # croplands
               'cc0013', # urban
               '33280d', # crop mosaic
               'd7cdcc', # snow and ice
               'f7e084', # barren
               '6f6f6f'  # tundra
              ]

In [ ]:
# Display the input and the classification.
Input_token = Input.getMapId({'bands': ['B3', 'B2', 'B1'],
                              'max': 0.4
                             })

classified_token = classified.getMapId({'palette': igbpPalette,
                                        'min': 0, 'max': 17
                                       })

Mapdisplay([ 37.7295,-122.3942], {'Landsat':Input_token,
                                  'LandUse':classified_token},
           zoom_start=10
          )
# Mapdisplay([ 37.7295,-122.3942], {'Landsat':Input_token}, zoom_start=10)